In [1]:
import pandas as pd
import janitor
import os
print(os.getcwd())

/Users/benjaminfrizzell/Documents/MDS/DSCI-532_2025_8_rental-issue-tracker


[Github Repo Link](https://github.com/UBC-MDS/DSCI-532_2025_8_rental-issue-tracker)

## 1. Load Data

In [2]:
# load data
data_path = 'data/raw/'
rental_issue_fname = 'rental-standards-current-issues.csv'
property_issue_fname = 'property-tax-report-2025.csv'
output_dir = 'data/clean/'

tax_report_cols = ['ZONING_CLASSIFICATION','FROM_CIVIC_NUMBER','TO_CIVIC_NUMBER','STREET_NAME','CURRENT_LAND_VALUE','YEAR_BUILT']

rental_issue_df = pd.read_csv(data_path+rental_issue_fname,sep=';')
property_tax_df = pd.read_csv(data_path+property_issue_fname,sep=',',usecols=tax_report_cols)

In [3]:
property_tax_df.head(3)

,ZONING_CLASSIFICATION,FROM_CIVIC_NUMBER,TO_CIVIC_NUMBER,STREET_NAME,CURRENT_LAND_VALUE,YEAR_BUILT
0,Comprehensive Development,606,8131.0,NUNAVUT LANE,490000.0,2016.0
1,Comprehensive Development,605,1308.0,HORNBY ST,364000.0,2014.0
2,Comprehensive Development,1602,1788.0,COLUMBIA ST,802000.0,2017.0


In [4]:
rental_issue_df.head(3)

,BUSINESSOPERATOR,DetailURL,StreetNumber,Street,TOTALOUTSTANDING,TotalUnits,Geom,Geo Local Area,geo_point_2d
0,Heritage House Holdings Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,455,ABBOTT STREET,4,109.0,"{""coordinates"": [-123.10777511, 49.28139465], ...",Downtown,"49.28139465, -123.10777511"
1,Provincial Rental Housing Corporation,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,525,ABBOTT STREET,1,108.0,"{""coordinates"": [-123.1081018, 49.28083259], ""...",Downtown,"49.28083259, -123.1081018"
2,0707892 BC Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,320,ABBOTT STREET,1,60.0,"{""coordinates"": [-123.10664756, 49.28261413], ...",Downtown,"49.28261413, -123.10664756"


## 2. Data Cleaning

### Cleaning `rental_issue_df`

In [5]:
# DATA CLEANING

# clean up names
rental_issue_df = (
    rental_issue_df
    .rename(columns={
        'BUSINESSOPERATOR':'business_operator',
        'DetailURL':'detail_url',
        'StreetNumber':'street_number',
        'TotalUnits':'total_units',
        'TOTALOUTSTANDING':'total_outstanding'})
    .clean_names()
    .drop(columns = ['geom'])
)

In [6]:
# separate geom into latitude and longitude columns
get_lat = lambda x: float(x.split(sep=', ')[0]) if isinstance(x,str) else None
get_long = lambda x: float(x.split(sep=', ')[1]) if isinstance(x,str) else None

rental_issue_df['lat'] = rental_issue_df['geo_point_2d'].apply(get_lat)
rental_issue_df['long'] = rental_issue_df['geo_point_2d'].apply(get_long)
rental_issue_df = rental_issue_df.drop(columns='geo_point_2d')

In [7]:
rental_issue_df.isna().sum()

business_operator    1
detail_url           0
street_number        0
street               0
total_outstanding    0
total_units          1
geo_local_area       6
lat                  6
long                 6
dtype: int64

In [8]:
# dropping rows with missing entries
rental_issue_df['business_operator'] = rental_issue_df['business_operator'].fillna('Unknown')
rental_issue_df = rental_issue_df.dropna()

In [9]:
# replace full street names with abbreviations
street_abbreviations = {
    'STREET': 'ST',
    'AVENUE': 'AVE',
    'BOULEVARD': 'BLVD',
    'ROAD': 'RD',
    'DRIVE': 'DR'
}
rental_issue_df['street'] = rental_issue_df['street'].replace(street_abbreviations,regex=True)

In [10]:
rental_issue_df['total_units'] = rental_issue_df['total_units'].astype(int)
rental_issue_df.head(3)

,business_operator,detail_url,street_number,street,total_outstanding,total_units,geo_local_area,lat,long
0,Heritage House Holdings Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,455,ABBOTT ST,4,109,Downtown,49.281395,-123.107775
1,Provincial Rental Housing Corporation,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,525,ABBOTT ST,1,108,Downtown,49.280833,-123.108102
2,0707892 BC Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,320,ABBOTT ST,1,60,Downtown,49.282614,-123.106648


In [11]:
rental_issue_df.to_csv(output_dir+'rental_issues_clean.csv')

### Cleaning `property_tax_df`

Note: addresses in this dataset are given as a range of addresses, so extra care is required when joining.
Some civic numbers have character values, but we can remove these as we know the only civic addresses in `rental_issues_df` are numeric, so they won't be useful here.


In [12]:
property_tax_df = property_tax_df.clean_names()


property_tax_df = property_tax_df[~property_tax_df['zoning_classification'].isin(['Comprehensive Development'])]
property_tax_df.head(3)

,zoning_classification,from_civic_number,to_civic_number,street_name,current_land_value,year_built
4,Residential,NaN,1535.0,5TH AVE E,1018000.0,2019.0
5,Historical Area,510,168.0,POWELL ST,385000.0,2009.0
6,Residential,NaN,2174.0,8TH AVE W,1547000.0,2008.0


If there is one entry in `from_civic_number` but none in `to_civic_number`, or vice versa, then the property only has one civic number. We will fill missing entries from one column with the other in each row, to help with joining later.

In [13]:
# convert to numeric typing, removing non-numeric civic addresses
property_tax_df['from_civic_number'] = pd.to_numeric(property_tax_df['from_civic_number'],errors='coerce')
property_tax_df['to_civic_number'] = pd.to_numeric(property_tax_df['to_civic_number'],errors='coerce')

# fill missing values with available civic numbers
property_tax_df['from_civic_number'] = property_tax_df['from_civic_number'].fillna(property_tax_df['to_civic_number'])
property_tax_df['to_civic_number'] = property_tax_df['to_civic_number'].fillna(property_tax_df['from_civic_number'])

In [14]:
# drop properties that are missing both civic addresses, or street name
property_tax_df = property_tax_df.dropna(subset=['from_civic_number','to_civic_number','street_name'])

# fill other missing entries with 'unknown'
property_tax_df['year_built'] = property_tax_df['year_built'].astype('Int64') # to Int64, to remove trailing zeros
property_tax_df['zoning_classification'] = property_tax_df['zoning_classification'].fillna('Unknown')

property_tax_df.isna().sum()

zoning_classification       0
from_civic_number           0
to_civic_number             0
street_name                 0
current_land_value       2557
year_built               5042
dtype: int64

In [15]:
# convert columns to appropriate types
property_tax_df['from_civic_number'] = property_tax_df['from_civic_number'].astype(int)
property_tax_df['to_civic_number'] = property_tax_df['to_civic_number'].astype(int)

In [16]:
property_tax_df.head(3)

,zoning_classification,from_civic_number,to_civic_number,street_name,current_land_value,year_built
4,Residential,1535,1535,5TH AVE E,1018000.0,2019
5,Historical Area,510,168,POWELL ST,385000.0,2009
6,Residential,2174,2174,8TH AVE W,1547000.0,2008


The last fix that needs to be made, is that some civic number ranges are out of order. According to the database schema, `from_civic_number` should be the first house in a range, and `to_civic_number` should be the last number in a range. We can fix this by swapping these two entries for rows where they are out of order.

In [17]:
def swap_civic_numbers(row):
    '''Swap unordered civic number range entries in the property tax dataframe'''
    if row['from_civic_number'] > row['to_civic_number']:
        return (row['to_civic_number'],row['from_civic_number'])
    else:
        return (row['from_civic_number'],row['to_civic_number'])

In [18]:
property_tax_df[['from_civic_number','to_civic_number']] = property_tax_df.apply(swap_civic_numbers,axis=1, result_type='expand')

In [19]:
property_tax_df

,zoning_classification,from_civic_number,to_civic_number,street_name,current_land_value,year_built
4,Residential,1535,1535,5TH AVE E,1018000.0,2019
5,Historical Area,168,510,POWELL ST,385000.0,2009
6,Residential,2174,2174,8TH AVE W,1547000.0,2008
7,Industrial,278,278,2ND AVE E,467000.0,2009
10,Residential Inclusive,1975,1975,SASAMAT ST,3852000.0,2012
...,...,...,...,...,...,...
224720,Residential,8726,8726,MONTCALM ST,4932000.0,1963
224721,Residential,102,8808,MONTCALM ST,341000.0,1980
224723,Residential Inclusive,7579,7579,MONTCALM ST,2098000.0,1944
224724,Residential Inclusive,7791,7791,CARTIER ST,2253000.0,1973


In [20]:
property_tax_df.to_csv(output_dir+'property_tax_clean.csv')

## 3. Joining DataFrames

Since the property tax dataframe gives civic addresses as a range, the initial intentt was to do the following operations:

1. Inner join `rental_issue_df` and `property_tax_df`
2. Filter for rows where `street_number` is between `from_civic_address` and `to_civic_address`

However, there are many overlapping civic number ranges which makes the property value ambiguous. To avoid making bad extrapolations, we'll restrict joining to only direct matches with `street_number` and `from_civic_number` or `to_civic_number`. Unfortunately there is little more we can do without more understanding of how the data is collected.

In [21]:
joined_df = pd.merge(
    rental_issue_df,property_tax_df,how='inner',
    left_on='street',right_on='street_name')

In [22]:
joined_df = joined_df[
    (joined_df['street_number'] == joined_df['from_civic_number']) |
    (joined_df['street_number'] == joined_df['to_civic_number'])]
joined_df

,business_operator,detail_url,street_number,street,total_outstanding,total_units,geo_local_area,lat,long,zoning_classification,from_civic_number,to_civic_number,street_name,current_land_value,year_built
115,0707892 BC Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,320,ABBOTT ST,1,60,Downtown,49.282614,-123.106648,Historical Area,320,320,ABBOTT ST,10272000.0,1911
214,Central City Foundation,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,404,ABBOTT ST,4,71,Downtown,49.281685,-123.107136,Historical Area,233,404,ABBOTT ST,388000.0,1911
425,0730235 BC Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,1683,ADANAC ST,7,21,Grandview-Woodland,49.277827,-123.070885,Residential,1683,1683,ADANAC ST,4733000.0,1994
872,Lily Jang,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,1785,ADANAC ST,1,5,Grandview-Woodland,49.277803,-123.068787,Residential,1785,1785,ADANAC ST,1945000.0,1952
1547,Entre Nous Femmes Housing Society,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,1656,ADANAC ST,4,46,Grandview-Woodland,49.277334,-123.072002,Residential,1656,1656,ADANAC ST,6118000.0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101657,York Street Properties Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,2275,YORK AVE,1,9,Kitsilano,49.271969,-123.156724,Residential,2275,2275,YORK AVE,2784000.0,1957
101983,2425 York Nominee Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,2425,YORK AVE,2,24,Kitsilano,49.272027,-123.160414,Residential,2425,2425,YORK AVE,5568000.0,1965
102180,Yorkshire Place Apt Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,2145,YORK AVE,1,43,Kitsilano,49.271935,-123.154015,Residential,2145,2145,YORK AVE,10944000.0,1979
102825,Martin & Susan Sally,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,2366,YORK AVE,3,11,Kitsilano,49.271494,-123.158893,Residential,2366,2366,YORK AVE,2784000.0,1960


In [23]:
joined_df = joined_df.drop(columns=['from_civic_number','to_civic_number','street_name'])
joined_df.head()

,business_operator,detail_url,street_number,street,total_outstanding,total_units,geo_local_area,lat,long,zoning_classification,current_land_value,year_built
115,0707892 BC Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,320,ABBOTT ST,1,60,Downtown,49.282614,-123.106648,Historical Area,10272000.0,1911
214,Central City Foundation,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,404,ABBOTT ST,4,71,Downtown,49.281685,-123.107136,Historical Area,388000.0,1911
425,0730235 BC Ltd,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,1683,ADANAC ST,7,21,Grandview-Woodland,49.277827,-123.070885,Residential,4733000.0,1994
872,Lily Jang,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,1785,ADANAC ST,1,5,Grandview-Woodland,49.277803,-123.068787,Residential,1945000.0,1952
1547,Entre Nous Femmes Housing Society,http://app.vancouver.ca/RPS_Net/Default.aspx?n...,1656,ADANAC ST,4,46,Grandview-Woodland,49.277334,-123.072002,Residential,6118000.0,<NA>


In [24]:
joined_df.to_csv(output_dir+'rentals_with_property_value.csv')

## Final Summary
- I've condensed the raw `property_tax_report_2025.csv` so it just includes columns we need for the dashboard without becoming too big.
- I've added cleaned dataframes for the property tax data and rental issues data in the `data/clean` directory. 
- All of the residences in the rental issues dataset that have matching property value records have been joined to one dataframe in `data/clean/rental_issues_with_property_values.csv`. There are only 198 records; this is because many properties had duplicate entries and multiple land values due to the way the property tax data is structured. Unfortunately  I think this is the best we can do without more understanding of the data collection process but it should be enough for what we need.